In [1]:
#import libs
import os, glob
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from datetime import date, timedelta
import warnings
warnings.simplefilter("ignore")

In [2]:
#Merge CSVs
path = "../data/raw/log"
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)

In [3]:
df = df_merged.copy()
df.head(2)

,,Severity,Name,Alarm Source,Occurred On (NT),Cleared On (NT),Type,Clearance Status,Alarm Duration
0,\t\t-,Minor,Task execution failure alarm,OSS,9/17/2021 19:39,1/22/2022 15:16,QoS,Cleared,3043 hours 36 minutes 45 seconds
1,\t\t-,Minor,Task execution failure alarm,OSS,9/18/2021 21:35,1/22/2022 15:16,QoS,Cleared,3017 hours 40 minutes 50 seconds


In [4]:
df = df[['Severity','Name','Alarm Source','Occurred On (NT)','Cleared On (NT)','Type','Alarm Duration']]
df.head(2)

,Severity,Name,Alarm Source,Occurred On (NT),Cleared On (NT),Type,Alarm Duration
0,Minor,Task execution failure alarm,OSS,9/17/2021 19:39,1/22/2022 15:16,QoS,3043 hours 36 minutes 45 seconds
1,Minor,Task execution failure alarm,OSS,9/18/2021 21:35,1/22/2022 15:16,QoS,3017 hours 40 minutes 50 seconds


In [5]:
#We Don't need OSS Alarms
df.drop(df[df['Alarm Source'] =='OSS'].index, inplace=True)

In [6]:
#Drop duplicates
df.drop_duplicates(inplace=True)

In [7]:
df['Occurred On (NT)'] = pd.to_datetime(df['Occurred On (NT)'])
df['Cleared On (NT)'] = pd.to_datetime(df['Cleared On (NT)'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6400938 entries, 40 to 7712598
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Severity          object        
 1   Name              object        
 2   Alarm Source      object        
 3   Occurred On (NT)  datetime64[ns]
 4   Cleared On (NT)   datetime64[ns]
 5   Type              object        
 6   Alarm Duration    object        
dtypes: datetime64[ns](2), object(5)
memory usage: 390.7+ MB


In [9]:
df['Alarm_Duration_Hour'] = (pd.to_datetime(df['Cleared On (NT)'])-pd.to_datetime(df['Occurred On (NT)']))/ pd.Timedelta(hours=1)

## Save File to parquet file

In [10]:
df.to_parquet('../data/processed/Alarms_data_processed.parquet')